In [16]:
import pandas as pd
import os

In [17]:
path_parquet = os.path.join('..','data', 'interim', 'Ensanut-data.parquet')
df = pd.read_parquet(path_parquet)
df['Sexo'] = df['Sexo'].astype('category')  # Convertir a categoría
df.dtypes

Folio                                   object
Edad                                     int64
Sexo                                  category
C_Entidad                                int64
Entidad                                 object
Fecha                           datetime64[ns]
Atentar_contras_si                    category
Depresion                             category
Tristeza                              category
Cuantos cigarrillos (numero)             Int64
Frecuencia emborrachar                category
dtype: object

In [18]:
valores_unicos = df.nunique() # Contar valores únicos en cada columna

print(valores_unicos) # Mostrar el resultado

Folio                           196525
Edad                               103
Sexo                                 2
C_Entidad                           32
Entidad                             32
Fecha                              709
Atentar_contras_si                   7
Depresion                            8
Tristeza                             9
Cuantos cigarrillos (numero)        66
Frecuencia emborrachar              44
dtype: int64


In [19]:
# Obtener todos los valores únicos de la columna "frecuencia"
valores_unicos_frecuencia = df['Frecuencia emborrachar'].unique()

# Imprimir los valores únicos
valores_unicos_frecuencia

[NaN, '1', '2', '8', '3', ..., '50', '48', '35', '21', '27']
Length: 45
Categories (44, object): ['0', '00', '01', '02', ..., '88', '9', '98', '99']

In [28]:
max_edad = df["Edad"].max()
max_edad

999

In [ ]:
pip install great_expectations

In [ ]:
pip install --upgrade great_expectations


In [ ]:
pip install pandera

In [31]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check

# Función personalizada para verificar la edad y generar un mensaje específico
def validar_edad(series):
    # Verifica que las edades estén entre 0 y 120
    if (series >= 0).all() and (series <= 120).all():
        return True
    else:
        # Imprime mensaje si alguna edad no cumple
        print("Hay que corregir la columna Edad: valores fuera del rango 0-120.")
        return False

# Definir el esquema de validación con la función personalizada
schema = DataFrameSchema({
    "Edad": Column(int, Check(validar_edad), nullable=False),
    "Sexo": Column(int, Check.isin([1, 2]), nullable=False),
    "C_Entidad": Column(int, Check.in_range(1, 32), nullable=False),
    "Entidad": Column(str, Check(lambda s: s.str.isalpha()), nullable=False),
    "Fecha": Column(pa.DateTime, nullable=True),
    "Atentar_contras_si": Column(float, Check.isin([1.0, None]), nullable=True),
    "Depresion": Column(float, Check.isin([1.0, None]), nullable=True),
    "Tristeza": Column(float, Check.isin([1.0, None]), nullable=True),
    "Cuantos cigarrillos (numero)": Column(float, Check.ge(0), nullable=True),
    "Frecuencia emborrachar": Column(float, Check.ge(0), nullable=True),
})

# Aplicar el esquema de validación
try:
    validated_df = schema.validate(df)
    print("Validación exitosa.")
except pa.errors.SchemaErrors as e:
    print("Errores de validación:", e)


Hay que corregir la columna Edad: valores fuera del rango 0-120.


SchemaError: Column 'Edad' failed series or dataframe validator 0: <Check validar_edad>